<a href="https://colab.research.google.com/github/RobinsonGo09/Calculadora/blob/master/ReconocimientoObjetos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importamos nuestro drive y le damos permiso a colab de ingresar.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Desde Colab importamos el parche cv2_imshow, para poder mostrar nuestra imagen.

In [ ]:
from google.colab.patches import cv2_imshow

# Importamos cv2 con la finalidad de poder utilizar algoritmos de visión artificial y numpy con la finalidad de realizar cálculos numéricos de forma eficiente en nuestro caso analizara matrices.



In [ ]:
import cv2
import numpy as np

# Cargamos el modelo de detección de objetos COCO 80 clases, y la máscara rcnn ( Red neuronal convolucional de detención de objetos ), para que los objetos que sean detectados agregarle una máscara de segmentación.

In [ ]:
rcnn = cv2.dnn.readNetFromTensorflow('/content/drive/MyDrive/frozen_inference_graph_coco.pb',
                                     '/content/drive/MyDrive/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt')

# Leemos la imagen

In [ ]:
img = cv2.imread('/content/drive/MyDrive/img1.jpeg')
alto, ancho, _ =  img.shape


# Generamos los colores

In [ ]:
colores = np.random.randint(0, 255, (80,3))

# Alistamos nuestra imagen

In [ ]:
blob = cv2.dnn.blobFromImage(img, swapRB = True) 

# Procesamos la imagen

In [ ]:
rcnn.setInput(blob)

# Extraemos los Rect y Mascaras

In [ ]:
info, masks = rcnn.forward(["detection_out_final", "detection_masks"])

# Extraemos la cantidad de objetos detectados

In [ ]:
contObject = info.shape[2]

# Iteramos sobre los objetos detectados

In [ ]:
for i in range(contObject):
      # Extraemos los rectangulos de los objetos
    inf = info[0,0,i]
    #print(inf)

    # Extraemos Clase
    clase = int(inf[1])
    #print(clase)

    # Extraemos puntaje
    puntaje = inf[2]

    # Filtro
    if puntaje < 0.7:
        continue

    # Coordenadas del Rectangulos para deteccion de objetos
    x = int(inf[3] * ancho)
    y = int(inf[4] * alto)
    x2 = int(inf[5] * ancho)
    y2 = int(inf[6] * alto)

    # Extraemos el tamaño de los objetos
    tamobj = img[y:y2, x:x2]
    tamalto, tamancho, _ = tamobj.shape
    #print(tamalto, tamancho)

    # Extraemos Mascara
    mask = masks[i, clase]
    mask = cv2.resize(mask, (tamancho, tamalto))

    # Establecemos un umbral
    _, mask = cv2.threshold(mask, 0.5, 255, cv2.THRESH_BINARY)
    mask = np.array(mask, np.uint8)
    #print(mask.shape)

    # Extraemos coordenadas de la mascara
    contornos, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Elegimos los colores
    color = colores[clase]
    r = int(color[0])
    g = int(color[1])
    b = int(color[2])

    # Iteramos los contornos
    for cont in contornos:
        # Dibujamos mascara
        cv2.fillPoly(tamobj, [cont], (r,g,b))
        # Dibujamos
        cv2.rectangle(img, (x, y), (x2, y2), (r, g, b), 3)

        #print(cont)
    # Mostramos
    cv2_imshow(img)
    
    # # Mascara
    # cv2.imshow('MASCARA', mask)
    cv2.waitKey(0)
